# Reacher Arm Project

![Screenshot of reacher arm environment](doc/BannerImage.png)

This is an implementation of the Deep Deterministic Policy Gradients Algorithm for training a two joint arm to keep its end effector within a moving target volume.

## Table of Contents
+ Environment Setup
+ Description of Algorithm
  - N-Step Bootstrapping
  - Prioritized Replay
  - NoisyNet Layers
+ Implementation of Algorithm
  - Hyperparameters
  - Helpers
  - Network Definition
  - Training Code
+ Training
+ Results
+ References

## Environment Setup

+ Follow instructions [here](https://github.com/udacity/Value-based-methods#dependencies) to set up the environment, *with the following changes:*
  - Before running `pip install .`, edit `Value-based-methods/python/requirements.txt` and remove the `torch==0.4.0` line
  - After running `pip install .`, run the appropriate PyTorch installation command for your system indicated [here](https://pytorch.org/get-started/locally/)
  - Continue following the instructions [here](https://github.com/udacity/Value-based-methods#dependencies) to their conclusion.
+ Download the appropriate Unity Environment for your platform:
  - [Linux](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Linux.zip)
  - [Mac OSX](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher.app.zip)
  - [Windows (32-bit)](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Windows_x86.zip)
  - [Windows (64-bit)](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Windows_x86_64.zip)
+ Place the Unity Environment zip file into any convenient directory, and unzip the file.

### Imports and references
Run the following code cell at every kernel instance start-up to bring implementation dependencies into the notebook namespace, and identify the path to the simulated environment executable.

In [6]:
from unityagents import UnityEnvironment
from collections import namedtuple, deque
from math import isnan
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

# Set to the path to simulated environment executable on system.
env_location = \
"C:/Projects/UdacityRLp2/Reacher_Windows_x86_64/Reacher_Windows_x86_64/Reacher.exe"

## Description of Algorithm

The Deep Deterministic Policy Gradient (DDPG) algorithm extends the application of Q-Learning methods to action spaces with continuously valued dimensions [[1]](#References).  There are two networks involved, a Policy Network and Action Value (Q) network.  During learning, the Policy Network generates an action according to the input state, and both this action and the state are supplied to the Q network as input.  The Q network outputs a single action value, and the gradient of this value with respect to the parameters of the Policy Network are used to nudge the policy towards one with a higher action value (by gradient ascent). <br><br> 
In typical Deep Q Learning (DQN), each action has a corresponding output from the Q network, but this representation of the Action Value Function $Q(s,a)$ does not naturally accomodate continuously-valued actions.  The primary difference of DDPG with respect to DQN, is that actions are instead explicit, continuously valued inputs to the Action Value function approximation.  This allows closed-form computation of the gradient of $Q(s,a)$ with respect to changes in magnitudes of the continuously-valued action variables.

The basic DDPG algorithm reads as follows (from [[1]](#References)): <br>
![DDPG Algorithm](doc/DDPG_alg.png) <br>

This implementation incorporates the following improvements:

### N-Step Bootstrapping

The hyperparameter `n_step_order`, determines the value of $n$ in the following alternative Bellman Update target, replacing the definition for $y_i$ in the algorithm above:

$$y_i = r_i + \gamma r_{i+1} + \gamma^2 r_{i+2} + ... + \gamma^n r_{i+n} + \gamma^{n+1} Q'(s_{i+n+1},\mu '(s_{i+n+1}|\theta^{\mu '})|\theta^{Q'})$$

N-Step Bootstrapping increases the relative weight of sampled rewards from the environment, compared to rewards estimated by the Action Value function $Q(s,a)$. Anecdotally, this seems to assist action values propagating backwards in time and through 'bottlenecks' where most nearby states have comparatively low State Values.  So essentially, initial learning can be faster, and some connections may be made that would otherwise take an unacceptably long time to be made without N-Step Bootstrapping.  However, real rewards are stochastic, and an atypically bad or good run of events will  more readily propagate through a Q network with N-Step Bootstrapping.  If an agent quickly changes its behavior between simple, regimented approaches, it is possible the `n_step_order` value in use is too high for the agent's environment.

### Prioritized Replay
The algorithm will periodically switch between exploration and learning phases.  <br><br>During exploration phases, state transition tuples $(S_t,a_t,r_t,S_{t+1})$ will be collected, transformed to *n-step* transition events via an accumulation buffer, and stored in a prioritized experience buffer. 
<br><br>
During learning phases, transition events sampled from the prioritized experience buffer will be used to optimize the parameters of the Policy and Q networks.  Like in [[2]](#References), the probability of utilizing a transition $T$ from the experience buffer is consistent with the proportionality relation: <br><br>
$$p_T \varpropto (Loss)^{\alpha}, \alpha \in [0,\infty)$$
<br>
The hyperparameter $\alpha$ allows tuning of the degree to which the probability of selection is affected by loss magnitude [[2]](#References).
<br><br>Qualitatively, the $Loss$ in this context is proportional to how inconsistent the parameterized model's prediction is with a prediction that uses actual rewards sampled from the environment.  See the implementation section for detail on how the loss is computed.

### NoisyNet Layers
For the Policy Net, noisy linear layers will be used instead of additive noise at the output.  Such layers have learnable noise parameters, so the noise level can be adaptively reduced as a deterministic policy develops (See [[3]](#References)).

## Implementation of Algorithm

### Hyperparameters

#### Environment
`state_dim`: Dimension of the observable state space<br>
`act_dim`: Dimension of the action space for each agent <br>
`num_agnt`: Number of agents in the environment

#### Network Models
`pol_hid_num`: Number of hidden layers in the Policy Network<br>
`pol_hid_size`: Number of neurons in each hidden layer of the Policy Network<br>
`noise_init`: Initial magnitude of the noise parameters in the NoisyNet layers<br>
`q_hid_num`: Number of hidden layers in the Q Network<br>
`q_hid_size`: Number of neurons in each hidden layer of the Q Network<br>

#### Reward Parameters
`gamma`: Discount factor per step for rewards<br>
`n_step_order`: Number of reward steps to directly incorporate into Bellman Update estimate<br>

#### Replay Parameters
`buf_life`: Buffer will be reset every this many samples<br>
`buf_min_size`: Learning will not be allowed unless replay buffer has this many experiences, to avoid overfitting<br>
`alpha`: Prioritization strength factor, see [[2]](#References)<br>
`beta`: Importance sampling correction coefficient, see [[2]](#References)<br>

#### Optimization Parameters
`pol_lr`: Learning rate for Policy Network optimizer<br>
`q_lr`: Learning rate for Q Network optimizer<br>
`lr_int`: Number of environment steps between each learning phase<br>
`lr_stps`: How many learning steps are applied during each learning phase<br>
`batch_size`: How many experiences are processed by each agent for each learning step<br>
`p_tau`: Soft update factor for target Policy Network, applied once every learning step<br>
`q_tau`: Soft update factor for the Q Network, applied once every learning step<br>

#### Training Parameters
`max_eps`: Maximum number of episodes for which to train<br>
`avg_wnd_len`: Length (in episodes) of running average buffer for reported performance<br>
`rprt_int`: Number of episodes between prints of performance<br>
`slv_thresh`: Minimum average score constituting solution of environment, the achievement of which will end the training run

In [3]:
class DDPG_Hyperparameters():
    def __init__(self,
                 state_dim=33,
                 act_dim=4,
                 num_agnt=20,
                 pol_hid_num=2,
                 pol_hid_size=300,
                 noise_init=0.1,
                 q_hid_num=2,
                 q_hid_size=300,
                 gamma=0.98,
                 n_step_order=5,
                 buf_life=2500,
                 buf_min_size=500,
                 alpha=0.6,
                 beta=1.0,
                 pol_lr=1e-5,
                 q_lr=1e-5,
                 lr_int=300,
                 lr_stps=500,
                 batch_size=16,
                 p_tau=4e-4,
                 q_tau=4e-4,
                 max_eps=2000,
                 avg_wnd_len=100,
                 rprt_int=5,
                 slv_thresh=30):
        self.state_dim, self.act_dim, self.num_agnt = state_dim, act_dim, num_agnt
        self.pol_hid_num, self.pol_hid_size, self.noise_init = pol_hid_num, pol_hid_size, noise_init
        self.q_hid_num, self.q_hid_size, self.gamma = q_hid_num, q_hid_size, gamma
        self.n_step_order, self.buf_life, self.buf_min_size = n_step_order, buf_life, buf_min_size
        self.alpha, self.beta, self.pol_lr = alpha, beta, pol_lr
        self.q_lr, self.lr_int, self.lr_stps = q_lr, lr_int, lr_stps
        self.batch_size, self.p_tau, self.q_tau = batch_size, p_tau, q_tau
        self.max_eps, self.avg_wnd_len, self.rprt_int = max_eps, avg_wnd_len, rprt_int
        self.slv_thresh = slv_thresh

def_hyp = DDPG_Hyperparameters()

### Helpers

In [7]:
# Debugging function
def tensor_check(input,desc,exp_size):
    if torch.any(torch.isnan(input)):
        print(f'NaNs in {desc}:')
        print(input)
    if torch.any(torch.isinf(input)):
        print(f'Inf in {desc}:')
        print(input)
    if not (input.size() == torch.size(exp_size)):
        print(f'{desc} has size {tuple(input.size())}, not {exp_size} expected')

# Copied from the Lunar Lander dqn_agent.py file of the Udacity repo for course
def soft_update(local_model, target_model, tau):
    """Soft update model parameters.
    θ_target = τ*θ_local + (1 - τ)*θ_target

    Params
    ======
        local_model (PyTorch model): weights will be copied from
        target_model (PyTorch model): weights will be copied to
        tau (float): interpolation parameter 
    """
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
        target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

# Object that represents an experience in the experience buffer or a non-leaf node of the sum tree
# Experience tuples are stored in the self.data attribute
# Based on code in Reference [4]
class SumTreeNode():
    
    def __init__(self,data=None,p_i=0):
        self.data = data
        self.p_i = p_i
        self.parent = None
        self.left_child = None
        self.right_child = None
    
    def update_p(self, delta_p):
        self.p_i += delta_p
        if self.parent is not None:
            self.parent.update_p(delta_p)
    
    def attach_child(self,child):
        if self.data is None:    # Not a leaf node
            if self.left_child is None:    # No children, become leaf with cloned data
                self.data = child.data
                self.update_p(child.p_i - self.p_i)
            else:    # Non-leaf node, attach to lower p_i side
                if self.left_child.p_i < self.right_child.p_i:
                    delegate_node = self.left_child
                else:
                    delegate_node = self.right_child
                delegate_node.attach_child(child)
        else:    # self is a leaf-node.  Clone self.data into new child, become non-leaf
            self.left_child = SumTreeNode(self.data,self.p_i)
            self.data = None
            self.right_child = child
            self.left_child.parent, self.right_child.parent = self, self     
            self.update_p((self.left_child.p_i + self.right_child.p_i)- self.p_i)
    
    def weighted_retrieve(self,p_samp):
        if self.data is not None: # must be a leaf-node
            return self
        else:
            if self.left_child.p_i >= p_samp:
                return self.left_child.weighted_retrieve(p_samp)
            else:
                return self.right_child.weighted_retrieve(p_samp - self.left_child.p_i)
        
# Experience aggregate
Experience = namedtuple('Experience',['state','action','reward','last_state'])
        
class PrioritizedReplayBuffer():
    
    def __init__(self,hyp):
        self.buf_life = hyp.buf_life
        self.alpha = hyp.alpha
        self.store = SumTreeNode()
        self.sample_count = 0
        self.exp_count = 0
        self.beta = hyp.beta
    
    def __len__(self):
        return self.exp_count
    
    def add_experience(self, experience, loss):
        new_p_i = pow(loss, self.alpha)
        self.store.attach_child(SumTreeNode(experience, new_p_i))
        self.exp_count += 1
            
    def sample(self,batch_size):
        sample_keys = (np.random.rand(batch_size)*self.store.p_i).tolist()
        samples = ([self.store.weighted_retrieve(p_samp) for p_samp in sample_keys],
                   (self.exp_count,self.beta,self.store.p_i))
        self.sample_count += 1
        if (self.sample_count >= self.buffer_life):
            self.sample_count = 0
            self.store = SumTreeNode()
            self.exp_count = 0
            print ('Flushed replay buffer!')
        return samples
    
# Circular buffer for generation of n_step rewards
class MultistepBuffer():
    def __init__(self,hyp):
        self.n_step_order = hyp.n_step_order
        self.store = deque(maxlen = hyp.n_step_order + 1)
        self.gamma = hyp.gamma
    
    def add_experience(self,exp):
        self.store.append(exp)
    
    def ready(self):
        return len(self.store) == (self.n_step_order + 1)
    
    def get_n_step_experience(self):
        out_state = self.store[0].state
        out_action = self.store[0].action
        out_reward = \
            sum([((self.store[i].reward) * pow(self.gamma,i)) for i in range(self.n_step_order)])
        out_final_state = self.store[-1].state
        return SumTreeNode(Experience(out_state, out_action, out_reward, out_final_state),p_i=1)
    
# Logger for running average
class PerformanceLogger():
    def __init__(self,avg_wnd_len=100,starting_scores=None):
        self.avg_wnd_len = avg_wnd_len
        self.scores = starting_scores if starting_scores is not None else []
        self.internal_run_avg = 0
        
    def add_score(self,score):
        self.scores.append(score)
        self.internal_run_avg += score / self.avg_wnd_len
        # Remove tail of running average
        if len(self.scores) > self.avg_wnd_len:
            self.internal_run_avg -= self.scores[-(self.avg_wnd_len + 1)] / self.avg_wnd_len
    
    def has_full_window(self):
        return len(self.scores) >= self.avg_wnd_len
    
    def run_avg(self):
        return self.internal_run_avg

### Network Definition

In [ ]:
class NoisyLinear(nn.Module):
    def __init__(self,in_dim,out_dim,noise_init=0.05):
        super(NoisyLinear,self).__init__()
        self.deterministic_linear = nn.Linear(in_dim,out_dim)
        self.noisy_weights = \
            nn.Parameter(noise_init*torch.ones(in_dim,out_dim,dtype=torch.float32))
        self.noisy_bias = nn.Parameter(noise_init*torch.ones(1,out_dim,dtype=torch.float32))
    
    def noise_transform(self,x):
        # See section 3(b) of reference [3]
        tensor_check(x,'Input to noise_transform',tuple(x.size()))
        x = torch.mul(torch.sgn(x),torch.sqrt(torch.abs(x)))
        tensor_check(x,'Output of noise_transform',tuple(x.size()))
        return x
    
    def forward(self,x):
        # Generate factorized gaussian noise, clamping to +/- 5 sigma
        in_dim_noise = \
            self.noise_transform( \
                torch.clamp( \
                    torch.randn((self.noisy_weights.size(dim=0),1),device=x.device),-5.0,5.0))
        out_dim_noise = \
            self.noise_transform( \
                torch.clamp( \
                    torch.randn((1,self.noisy_weights.size(dim=1)),device=x.device),-5.0,5.0))
        
        weight_noise = torch.mul(self.noisy_weights,torch.matmul(in_dim_noise,out_dim_noise))
        bias_noise = torch.mul(self.noisy_bias,out_dim_noise)
    
        x = x.float()
        return self.deterministic_linear(x) + torch.matmul(x,weight_noise) + bias_noise

# Generic MLP
class DDPG_Subnet(nn.Module):
    def __init__(self, 
                 in_dim, 
                 out_dim, 
                 hid_size, 
                 num_hid,
                 squish_output=False,
                 noisy=False, 
                 noise_init=0.05):
        super(DDPG_Subnet,self).__init__()
        
        layers = []
        if noisy:
            layers.append(NoisyLinear(in_dim, hid_size, noise_init))
        else:
            layers.append(nn.Linear(in_dim, hid_size))
        layers.append(nn.LeakyReLU(negative_slope=0.05))
        for i in range(num_hid-1):
            if noisy:
                layers.append(NoisyLinear(hid_size, hid_size, noise_init))
            else:
                layers.append(nn.Linear(hid_size, hid_size))
            layers.append(nn.LeakyReLU(negative_slope=0.05))
        if noisy:
            layers.append(NoisyLinear(hid_size, out_dim, noise_init))
        else:
            layers.append(nn.Linear(hid_size, out_dim))
        
        self.reg_layers = nn.Sequential(*layers)
        self.squish_output = squish_output
        
    def forward(self,x):
        x = self.reg_layers(x)
        if self.squish_output:
            x = tanh(x)
        return x
    
# Instantiate
pol_net = DDPG_Subnet(def_hyp.state_dim,
                      def_hyp.act_dim,
                      def_hyp.pol_hid_size,
                      def_hyp.pol_hid_num,
                      True,
                      True,
                      def_hyp.noise_init)

q_net = DDPG_Subnet(def_hyp.state_dim + def_hyp.act_dim,
                    1,
                    def_hyp.q_hid_size,
                    def_hyp.q_hid_num,
                    False,
                    False,
                    0)

### Training Code

## References
[1] Lillicrap et. al., Continuous control with deep reinforcement learning, [arXiv:1509.02971](https://arxiv.org/abs/1509.02971) <br>
[2] Schaul et. al., Prioritized Experience Replay, [arXiv:1511.05952](https://arxiv.org/abs/1511.05952)<br>
[3] Fortunato et. al., Noisy Networks for Exploration, [arXiv:1706.10295](https://arxiv.org/abs/1706.10295)<br>
[4] http://www.sefidian.com/2022/09/09/sumtree-data-structure-for-prioritized-experience-replay-per-explained-with-python-code/<br>
